# WeatherPy

---

## Starter Code to Generate Random Geographic Coordinates and a List of Cities

In [110]:
# Import Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint   # Potentially not needed later!!!

# `Citipy` determines cities based on latitude & longitude
from citipy import citipy   # Install using `pip install citipy`

# Import OpenWeatherMap API Key
from api_keys import weather_api_key

### Generate the Cities List by Using the `citipy` Library

In [119]:
# Empty list for holding the latitude and longitude combinations
lat_lngs = []

# Empty list for holding the cities names
cities = []

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size = 100) # Please change back `size = 1500`!!!
lngs = np.random.uniform(lng_range[0], lng_range[1], size = 100) # Please change back `size = 1500`!!!
lat_lngs = list(zip(lats, lngs))    # Correction to allow for later access of coordinates
                                    # Potentially not needed!!! if actually correct coordinates
                                    # can be found

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name

    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(f"Number of cities in the list: {len(cities)}")

Number of cities in the list: 78


---

## Requirement 1: Create Plots to Showcase the Relationship Between Weather Variables and Latitude

### Use the OpenWeatherMap API to retrieve weather data from the cities list generated in the started code

In [ ]:
# Set the API base URL
# https://api.openweathermap.org/data/2.5/weather?q=London,uk&APPID=4312e8b42443141d8193bff21246737e
# https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&exclude={part}&appid={API key}
base_url = f'https://api.openweathermap.org/data/2.5/weather?appid={weather_api_key}'

# Define an empty list to fetch the weather data for each city
city_data = []

# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1    # Correction from starter_code: `record_count = 0`

    # Create endpoint URL with each city
    city_url = f'{base_url}&q={city}'

    # Log the url, record, and set numbers
    # print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))
    print(f'Processing Record {record_count} of Set {set_count} | {city}')

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        response = requests.get(city_url).json()

        # Parse the JSON and retrieve data
        city_weather = response['weather']

        # Parse out latitude, longitude, max temp, humidity, cloudiness, wind speed, country, and date
        city_lat = response['coord']['lat']
        city_lng = response['coord']['lon']
        city_max_temp = response['main']['temp_max']
        city_humidity = response['main']['humidity']
        city_clouds = response['clouds']
        city_wind = response['wind']
        city_country = response['sys']['country']
        city_date = response['dt']

        # Append the City information into city_data list
        city_data.append({"City": city,
                        "Lat": city_lat,
                        "Lng": city_lng,
                        "Max Temp": city_max_temp,
                        "Humidity": city_humidity,
                        "Cloudiness": city_clouds,
                        "Wind Speed": city_wind,
                        "Country": city_country,
                        "Date": city_date})

    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass

    # pause to avoid rate limiting
    time.sleep(1)

# Indicate that Data Loading is complete
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | yuzhno-kurilsk
Processing Record 2 of Set 1 | zakamensk
Processing Record 3 of Set 1 | vanju-mare
Processing Record 4 of Set 1 | bethel
Processing Record 5 of Set 1 | papatowai
Processing Record 6 of Set 1 | mouila
Processing Record 7 of Set 1 | soligalich
Processing Record 8 of Set 1 | iqaluit
Processing Record 9 of Set 1 | puerto ayora
Processing Record 10 of Set 1 | blackmans bay
Processing Record 11 of Set 1 | wailua homesteads
Processing Record 12 of Set 1 | codrington
Processing Record 13 of Set 1 | bredasdorp
Processing Record 14 of Set 1 | longyearbyen
Processing Record 15 of Set 1 | afaahiti
Processing Record 16 of Set 1 | constantia
Processing Record 17 of Set 1 | ad dilam
Processing Record 18 of Set 1 | navarre
Processing Record 19 of Set 1 | flying fish cove
Processing Record 20 of Set 1 | lae
Processing Record 21 of Set 1 | nova olinda do norte
Processing Record 22 of Set 1 | puerto 

In [121]:
# Convert the cities weather data into a Pandas DataFrame
city_data_df = pd.DataFrame(city_data)

# Show Record Count
city_data_df.count()

City          76
Lat           76
Lng           76
Max Temp      76
Humidity      76
Cloudiness    76
Wind Speed    76
Country       76
Date          76
dtype: int64

In [122]:
# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,yuzhno-kurilsk,44.0306,145.8556,271.08,57,{'all': 78},"{'speed': 9.34, 'deg': 300, 'gust': 12.06}",RU,1735465833
1,zakamensk,50.3741,103.2863,255.81,91,{'all': 45},"{'speed': 2.07, 'deg': 242, 'gust': 1.63}",RU,1735465833
2,vanju-mare,44.4253,22.8697,277.43,64,{'all': 12},"{'speed': 1.34, 'deg': 261, 'gust': 0.98}",RO,1735465833
3,bethel,41.3712,-73.4140,279.82,90,{'all': 100},"{'speed': 0.89, 'deg': 265, 'gust': 0.89}",US,1735465777
4,papatowai,-46.5619,169.4708,286.38,98,{'all': 100},"{'speed': 1.83, 'deg': 225, 'gust': 2.29}",NZ,1735465833


In [ ]:
# Export the City_Data into a csv
city_data_df.to_csv("output_data/cities.csv", index_label="City_ID")

In [ ]:
# Read saved data
city_data_df = pd.read_csv("output_data/cities.csv", index_col="City_ID")

# Display sample data
city_data_df.head()

### Create the Scatter Plots Requested

#### Latitude Vs. Temperature

In [ ]:
# Build scatter plot for latitude vs. temperature
# YOUR CODE HERE

# Incorporate the other graph properties
# YOUR CODE HERE

# Save the figure
plt.savefig("output_data/Fig1.png")

# Show plot
plt.show()

#### Latitude Vs. Humidity

In [ ]:
# Build the scatter plots for latitude vs. humidity
# YOUR CODE HERE

# Incorporate the other graph properties
# YOUR CODE HERE

# Save the figure
plt.savefig("output_data/Fig2.png")

# Show plot
plt.show()

#### Latitude Vs. Cloudiness

In [ ]:
# Build the scatter plots for latitude vs. cloudiness
# YOUR CODE HERE

# Incorporate the other graph properties
# YOUR CODE HERE

# Save the figure
plt.savefig("output_data/Fig3.png")

# Show plot
plt.show()

#### Latitude vs. Wind Speed Plot

In [ ]:
# Build the scatter plots for latitude vs. wind speed
# YOUR CODE HERE

# Incorporate the other graph properties
# YOUR CODE HERE

# Save the figure
plt.savefig("output_data/Fig4.png")

# Show plot
plt.show()

---

## Requirement 2: Compute Linear Regression for Each Relationship


In [ ]:
# Define a function to create Linear Regression plots
# YOUR CODE HERE

In [ ]:
# Create a DataFrame with the Northern Hemisphere data (Latitude >= 0)
# YOUR CODE HERE

# Display sample data
northern_hemi_df.head()

In [ ]:
# Create a DataFrame with the Southern Hemisphere data (Latitude < 0)
# YOUR CODE HERE

# Display sample data
southern_hemi_df.head()

###  Temperature vs. Latitude Linear Regression Plot

In [ ]:
# Linear regression on Northern Hemisphere
# YOUR CODE HERE

In [ ]:
# Linear regression on Southern Hemisphere
# YOUR CODE HERE

**Discussion about the linear relationship:** YOUR RESPONSE HERE

### Humidity vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
# YOUR CODE HERE

In [ ]:
# Southern Hemisphere
# YOUR CODE HERE

**Discussion about the linear relationship:** YOUR RESPONSE HERE

### Cloudiness vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
# YOUR CODE HERE

In [ ]:
# Southern Hemisphere
# YOUR CODE HERE

**Discussion about the linear relationship:** YOUR RESPONSE HERE

### Wind Speed vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
# YOUR CODE HERE

In [ ]:
# Southern Hemisphere
# YOUR CODE HERE

**Discussion about the linear relationship:** YOUR RESPONSE HERE